In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.io import loadmat as loadmat

%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
rootfolder = ".."

Useful function for the plot


In [ ]:
def display_band(X, par, epsi=0.001, col="k"):
    x1 = np.min(X[0, :])
    x2 = np.max(X[0, :])
    y1 = np.min(X[1, :])
    y2 = np.max(X[1, :])

    # extend the line for the k# of its lenght
    k = 0.01
    dx = x2 - x1
    dy = y2 - y1
    l = np.sqrt(dx**2 + dy**2)
    x1 = x1 - k * l
    x2 = x2 + k * l
    y1 = y1 - k * l
    y2 = y2 + k * l
    ##
    if np.abs(par[1]) < 2e-2:
        # disp('vertical line')
        a = np.vstack([-(par[1] * y1 + par[2]) / par[0], y1])
        b = np.vstack([-(par[1] * y2 + par[2]) / par[0], y2])
    else:
        a = np.vstack([x1, -(par[0] * x1 + par[2]) / par[1]])
        b = np.vstack([x2, -(par[0] * x2 + par[2]) / par[1]])

    if np.abs(par[0]) < 1e-5:
        v = np.array([0, 1])
    else:
        v = np.array([1, par[1] / par[0]])  # direction perpendicular to the line;

    v = v / np.sqrt(np.sum(v**2))
    # corners of the bands
    v = v[:, np.newaxis]
    a1 = a - epsi * v
    a2 = a + epsi * v
    b1 = b - epsi * v
    b2 = b + epsi * v

    px = np.array([a1[0], b1[0], b2[0], a2[0]]).squeeze()
    py = np.array([a1[1], b1[1], b2[1], a2[1]]).squeeze()
    plt.tricontourf(px, py, np.ones_like(px), colors=col, alpha=0.5)
    plt.gca().set_aspect("equal")


def display_clust(X, G, cmap):
    G = np.asarray(G, dtype=int)
    switch_out = 1
    smb = "o"
    col = []
    num_clust = int(np.max(G))

    for i in range(1, num_clust + 1):
        id = G == i
        id = id.squeeze()
        x = X[0, id]
        y = X[1, id]
        if len(col) == 0:
            plt.scatter(x, y, 80, cmap[i, :], marker=smb, edgecolors="k", alpha=1)
        else:
            plt.scatter(x, y, 50, col, marker=smb, edgecolors=col, alpha=0.1)
        if switch_out == 1:
            id = G == 0
            id = id.squeeze()
            plt.scatter(
                X[0, id],
                X[1, id],
                50,
                [0.3, 0.3, 0.3],
                marker=smb,
                edgecolors=[0.2, 0.2, 0.2],
                alpha=1,
            )

Function that computes the residual between points and a line


In [ ]:
def res_line(X, M):
    if len(M.shape) > 1:
        num_lines = M.shape[1]
    else:
        num_lines = 1

    if num_lines == 1:
        d = np.abs(M[0] * X[0, :] + M[1] * X[1, :] + M[2])
    else:
        n = X.shape[1]
        d = np.zeros((n, num_lines))
        for i in range(num_lines):
            d[:, i] = np.abs(M[0, i] * X[0, :] + M[1, i] * X[1, :] + M[2, i])


Functions to add outlier to the dataset


In [ ]:
def addOutliersInBB(X, nOutliers, k=0.1):
    xmin = np.min(X[0, :])
    xmax = np.max(X[0, :])
    ymin = np.min(X[1, :])
    ymax = np.max(X[1, :])
    wx = xmax - xmin
    wy = ymax - ymin
    dx = k * wx
    dy = k * wy

    Y = np.hstack(
        (
            X,
            np.vstack(
                (
                    xmax
                    - xmin
                    + 2 * dx * np.random.uniform(size=(1, nOutliers))
                    + xmin
                    - dx,
                    ymax
                    - ymin
                    + 2 * dy * np.random.uniform(size=(1, nOutliers))
                    + ymin
                    - dy,
                )
            ),
        )
    )
    return Y


## Ransac

Use the implementation from the last lecture


In [ ]:
def fit_line_dlt(P):
    """
    Fit a line to 2D points using DLT
    P: 2xN array of points
    Returns: theta (line parameters [a,b,c] where ax+by+c=0), residuals, residual_error
    """
    n = P.shape[1]

    # Build the design matrix A for DLT
    A = np.zeros((n, 3))
    A[:, 0] = P[0, :]  # x coordinates
    A[:, 1] = P[1, :]  # y coordinates
    A[:, 2] = 1  # homogeneous coordinate

    # Solve using SVD
    U, S, Vt = np.linalg.svd(A)
    theta = Vt[-1, :]  # last row of V (smallest singular value)

    # Normalize so that a^2 + b^2 = 1
    norm = np.sqrt(theta[0] ** 2 + theta[1] ** 2)
    if norm > 0:
        theta = theta / norm

    # Compute residuals
    residuals = np.abs(A @ theta)
    residual_error = np.sum(residuals**2)

    return theta, residuals, residual_error


In [ ]:
# Simple RANSAC implementation
def simpleRANSAC(X, eps, cardmss=2):
    """
    RANSAC algorithm for line fitting
    X: 2xN array of points
    eps: inlier threshold
    cardmss: minimum number of points to fit model
    Returns: bestmodel, bestinliers
    """
    n = X.shape[1]
    max_iter = 1000
    best_consensus = 0
    bestmodel = None
    bestinliers = None

    # Probability of success
    p = 0.99

    iter_count = 0
    while iter_count < max_iter:
        # Random sample
        idx = np.random.choice(n, cardmss, replace=False)
        sample = X[:, idx]

        # Fit model to sample
        try:
            model, _, _ = fit_line_dlt(sample)

            # Compute residuals for all points
            residuals = res_line(X, model)

            # Find inliers
            inliers = residuals < eps
            consensus = np.sum(inliers)

            # Update best model if better
            if consensus > best_consensus:
                best_consensus = consensus
                bestmodel = model
                bestinliers = inliers

                # Update number of iterations
                w = best_consensus / n  # inlier ratio
                if w > 0 and w < 1:
                    max_iter = min(
                        max_iter, int(np.log(1 - p) / np.log(1 - w**cardmss))
                    )
        except:
            pass

        iter_count += 1

    return bestmodel, bestinliers


## Sequential Ransac

Prepare the dataset and set the parameters


In [ ]:
epsi = 0.02  # inlier threshold for ransac

# temp = loadmat(f'{rootfolder}/data/star5.mat')
temp = loadmat(f"{rootfolder}/data/stair4.mat")

X = temp["X"]  # data
G = temp["G"]  # ground truth or label (1: normal, 0: outlier)

# model specification
# functions to fit and compute residuals are already invoked in the function
cardmss = 2  # minimum number of points required to fit a model
kappa = 5  # number of models to be extracted by sequential ransac

#  plotting params
bw = 0.01
col = "b"
temp = loadmat(f"{rootfolder}/data/cmap.mat")
cmap = temp["cmap"]

# add outliers
nOut = 10

# add outlier and update groundtruth
X = addOutliersInBB(X, nOut)

# update the label, considering that X previous command has appended nOut otliers
G = np.hstack((G.flatten(), np.zeros(nOut)))


In [ ]:
plt.figure(1)
display_clust(X, G, cmap)
plt.title("Ground truth")
plt.gca().set_aspect("equal")
plt.show()

Implement the sequential ransac algorithm


In [ ]:
Y = X.copy()  # take a copy of X to be modified

# define cells to contain the fitted models
currentModel = []
currentInliers = []

E_min = 10  # minimum number of expected inliers for each

cnt = 0
while cnt < kappa:
    # find model with ransac
    model, inliers = simpleRANSAC(Y, epsi, cardmss)

    if model is None or np.sum(inliers) < E_min:
        print(f"No more models with at least {E_min} inliers found")
        break

    # save currentModel and currentInliers for visualization
    currentModel.append(model)
    currentInliers.append(inliers)

    # remove current inliers
    Y = Y[:, ~inliers]

    # update iterations
    cnt += 1

    print(f"Model {cnt}: {np.sum(inliers)} inliers found")

Show the iteration of Sequential Ransac


In [ ]:
Y = X.copy()
plt.gca().set_aspect("equal")
for i in range(len(currentModel)):
    plt.subplot(1, kappa, i + 1)

    # Plot all points
    plt.scatter(Y[0, :], Y[1, :], 20, "gray", alpha=0.5)

    # display a band for each model estimated
    display_band(Y, currentModel[i], epsi, cmap[i + 1, :])

    # plot inliers for this model
    inlier_indices = currentInliers[i]
    plt.scatter(
        Y[0, inlier_indices],
        Y[1, inlier_indices],
        50,
        cmap[i + 1, :],
        edgecolors="k",
        alpha=1,
    )

    # remove the inliers from Y
    Y = Y[:, ~currentInliers[i]]

    plt.title(f"Sequential RANSAC iteration {i + 1}")
    plt.gca().set_aspect("equal")
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))
Y = X.copy()
remaining_indices = np.ones(X.shape[1], dtype=bool)

for i in range(len(currentModel)):
    # display band for each model
    display_band(X, currentModel[i], epsi, cmap[i + 1, :])

    # find inliers in original data
    residuals = res_line(X, currentModel[i])
    inliers = (residuals < epsi) & remaining_indices

    # plot inliers
    plt.scatter(
        X[0, inliers], X[1, inliers], 50, cmap[i + 1, :], edgecolors="k", alpha=1
    )

    # mark these points as used
    remaining_indices[inliers] = False

# plot remaining points (outliers)
plt.scatter(
    X[0, remaining_indices],
    X[1, remaining_indices],
    50,
    [0.3, 0.3, 0.3],
    marker="o",
    edgecolors=[0.2, 0.2, 0.2],
    alpha=1,
)

plt.title("Sequential RANSAC - All Models")
plt.gca().set_aspect("equal")
plt.show()

TODO: Try different amount of outliers on both star and stair datasets


## TODO: MultiModel fitting on circle

Implement Ransac (thus run Sequential Ransac) to fit circles


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Circle

In [ ]:
def fit_circle(P):
    n = P.shape[1]

    # Build design matrix for circle fitting
    # (x-a)^2 + (y-b)^2 = r^2
    # x^2 - 2ax + a^2 + y^2 - 2by + b^2 = r^2
    # x^2 + y^2 - 2ax - 2by + (a^2 + b^2 - r^2) = 0

    A = np.zeros((n, 3))
    A[:, 0] = -2 * P[0, :]  # -2x
    A[:, 1] = -2 * P[1, :]  # -2y
    A[:, 2] = 1

    b = -(P[0, :] ** 2 + P[1, :] ** 2)

    # Solve using least squares
    params = np.linalg.lstsq(A, b, rcond=None)[0]

    center = params[:2]
    radius = np.sqrt(params[0] ** 2 + params[1] ** 2 - params[2])

    # Compute residuals
    distances = np.sqrt((P[0, :] - center[0]) ** 2 + (P[1, :] - center[1]) ** 2)
    residuals = np.abs(distances - radius)

    return center, radius, residuals


def res_circle(X, center, radius):
    distances = np.sqrt((X[0, :] - center[0]) ** 2 + (X[1, :] - center[1]) ** 2)
    return np.abs(distances - radius)


def circleRANSAC(X, eps, cardmss=3):
    n = X.shape[1]
    max_iter = 1000
    best_consensus = 0
    best_center = None
    best_radius = None
    bestinliers = None

    p = 0.99

    iter_count = 0
    while iter_count < max_iter:
        # Random sample
        idx = np.random.choice(n, cardmss, replace=False)
        sample = X[:, idx]

        try:
            # Fit circle to sample
            center, radius, _ = fit_circle(sample)

            if radius > 0 and radius < 10:  # Valid circle with reasonable radius
                # Compute residuals for all points
                residuals = res_circle(X, center, radius)

                # Find inliers
                inliers = residuals < eps
                consensus = np.sum(inliers)

                # Update best model if better
                if consensus > best_consensus:
                    best_consensus = consensus
                    best_center = center
                    best_radius = radius
                    bestinliers = inliers

                    # Update number of iterations
                    w = best_consensus / n
                    if w > 0 and w < 1:
                        max_iter = min(
                            max_iter, int(np.log(1 - p) / np.log(1 - w**cardmss))
                        )
        except:
            pass

        iter_count += 1

    return best_center, best_radius, bestinliers


def generate_circle_data(centers, radii, n_points_per_circle, noise_level=0.01):
    X = []
    G = []  # Ground truth labels

    for i, (center, radius) in enumerate(zip(centers, radii)):
        # Generate points on circle
        angles = np.random.uniform(0, 2 * np.pi, n_points_per_circle)
        x = (
            center[0]
            + radius * np.cos(angles)
            + np.random.normal(0, noise_level, n_points_per_circle)
        )
        y = (
            center[1]
            + radius * np.sin(angles)
            + np.random.normal(0, noise_level, n_points_per_circle)
        )

        X.append(np.vstack([x, y]))
        G.extend([i + 1] * n_points_per_circle)

    X = np.hstack(X)
    G = np.array(G)

    return X, G


def addOutliersInBB(X, nOutliers, k=0.1):
    xmin = np.min(X[0, :])
    xmax = np.max(X[0, :])
    ymin = np.min(X[1, :])
    ymax = np.max(X[1, :])
    wx = xmax - xmin
    wy = ymax - ymin
    dx = k * wx
    dy = k * wy

    Y = np.hstack(
        (
            X,
            np.vstack(
                (
                    xmax
                    - xmin
                    + 2 * dx * np.random.uniform(size=(1, nOutliers))
                    + xmin
                    - dx,
                    ymax
                    - ymin
                    + 2 * dy * np.random.uniform(size=(1, nOutliers))
                    + ymin
                    - dy,
                )
            ),
        )
    )
    return Y

In [ ]:
centers = [[0, 0], [3, 2], [-2, 3], [1, -3]]
radii = [1.5, 1.0, 0.8, 1.2]

In [ ]:
# Generate data
n_points_per_circle = 50
noise_level = 0.05
X, G = generate_circle_data(centers, radii, n_points_per_circle, noise_level)

# Add outliers
n_outliers = 30
X = addOutliersInBB(X, n_outliers, k=0.2)
G = np.hstack([G, np.zeros(n_outliers)])

# Plotting parameters
colors = plt.cm.tab10(np.linspace(0, 1, 10))

# Plot ground truth
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
for i in range(len(centers)):
    mask = G == i + 1
    plt.scatter(
        X[0, mask], X[1, mask], c=[colors[i]], s=30, alpha=0.7, label=f"Circle {i + 1}"
    )

outlier_mask = G == 0
plt.scatter(
    X[0, outlier_mask], X[1, outlier_mask], c="gray", s=20, alpha=0.5, label="Outliers"
)
plt.title("Ground Truth")
plt.xlabel("X")
plt.ylabel("Y")
plt.legend()
plt.axis("equal")
plt.grid(True, alpha=0.3)

In [ ]:
# Parameters
eps = 0.1  # inlier threshold
kappa = len(centers)  # number of circles to detect
E_min = 15  # minimum number of inliers

# Sequential RANSAC
Y = X.copy()
detected_circles = []
all_inliers = []

for i in range(kappa):
    print(f"\nIteration {i + 1}:")

    # Run RANSAC
    center, radius, inliers = circleRANSAC(Y, eps, cardmss=3)

    if center is None or np.sum(inliers) < E_min:
        print(f"No more circles with at least {E_min} inliers found")
        break

    print(
        f"  Circle found: center=({center[0]:.2f}, {center[1]:.2f}), radius={radius:.2f}"
    )
    print(f"  Number of inliers: {np.sum(inliers)}")

    # Store results
    detected_circles.append((center, radius))
    all_inliers.append(inliers)

    # Remove inliers
    Y = Y[:, ~inliers]

# Plot results
plt.subplot(1, 2, 2)
remaining_indices = np.ones(X.shape[1], dtype=bool)

for i, ((center, radius), inliers_in_Y) in enumerate(
    zip(detected_circles, all_inliers)
):
    # Find inliers in original data
    residuals = res_circle(X, center, radius)
    inliers = (residuals < eps) & remaining_indices

    # Plot points
    plt.scatter(
        X[0, inliers],
        X[1, inliers],
        c=[colors[i]],
        s=30,
        alpha=0.7,
        label=f"Detected {i + 1}",
    )

    # Draw circle
    circle = Circle(
        center, radius, fill=False, color=colors[i], linewidth=2, linestyle="--"
    )
    plt.gca().add_patch(circle)

    # Mark points as used
    remaining_indices[inliers] = False

# Plot remaining points
plt.scatter(
    X[0, remaining_indices],
    X[1, remaining_indices],
    c="gray",
    s=20,
    alpha=0.5,
    label="Unassigned",
)
plt.title("Sequential RANSAC Results")
plt.xlabel("X")
plt.ylabel("Y")
plt.legend()
plt.axis("equal")
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(15, 4))
Y = X.copy()
remaining_for_viz = np.ones(X.shape[1], dtype=bool)

for i in range(len(detected_circles)):
    plt.subplot(1, kappa, i + 1)

    # Plot all remaining points
    plt.scatter(Y[0, :], Y[1, :], c="lightgray", s=20, alpha=0.5)

    # Find and plot inliers for this circle
    center, radius = detected_circles[i]
    residuals = res_circle(X, center, radius)
    inliers = (residuals < eps) & remaining_for_viz

    plt.scatter(X[0, inliers], X[1, inliers], c=[colors[i]], s=50, alpha=0.8)

    # Draw detected circle
    circle = Circle(center, radius, fill=False, color=colors[i], linewidth=3)
    plt.gca().add_patch(circle)

    # Update for next iteration
    Y = Y[:, ~all_inliers[i]]
    remaining_for_viz[inliers] = False

    plt.title(f"Iteration {i + 1}\nInliers: {np.sum(inliers)}")
    plt.axis("equal")
    plt.grid(True, alpha=0.3)
    plt.xlim([-5, 6])
    plt.ylim([-5, 6])

plt.tight_layout()
plt.show()

# Performance evaluation
print("\n=== Performance Evaluation ===")
print(f"Number of circles in ground truth: {len(centers)}")
print(f"Number of circles detected: {len(detected_circles)}")
print(f"Total outliers: {n_outliers}")

# Compare detected circles with ground truth
print("\nDetected circles vs Ground truth:")
for i, (center, radius) in enumerate(detected_circles):
    print(f"\nDetected Circle {i + 1}:")
    print(f"  Center: ({center[0]:.2f}, {center[1]:.2f}), Radius: {radius:.2f}")

    # Find closest ground truth circle
    min_dist = float("inf")
    best_match = -1
    for j, (gt_center, gt_radius) in enumerate(zip(centers, radii)):
        dist = np.sqrt(
            (center[0] - gt_center[0]) ** 2 + (center[1] - gt_center[1]) ** 2
        )
        if dist < min_dist:
            min_dist = dist
            best_match = j

    if best_match >= 0:
        gt_center = centers[best_match]
        gt_radius = radii[best_match]
        center_error = min_dist
        radius_error = abs(radius - gt_radius)
        print(f"  Best match: Circle {best_match + 1}")
        print(f"  Center error: {center_error:.3f}")
        print(f"  Radius error: {radius_error:.3f}")

# Test with different noise levels
print("\n=== Testing with different noise levels ===")
noise_levels = [0.01, 0.05, 0.1, 0.15]
fig, axes = plt.subplots(1, len(noise_levels), figsize=(16, 4))

for idx, noise in enumerate(noise_levels):
    # Generate data with current noise level
    X_test, G_test = generate_circle_data(centers, radii, n_points_per_circle, noise)
    X_test = addOutliersInBB(X_test, n_outliers, k=0.2)

    # Run Sequential RANSAC
    Y_test = X_test.copy()
    detected = []

    for i in range(kappa):
        center, radius, inliers = circleRANSAC(Y_test, eps, cardmss=3)
        if center is None or np.sum(inliers) < E_min:
            break
        detected.append((center, radius))
        Y_test = Y_test[:, ~inliers]

    # Plot
    ax = axes[idx]
    ax.scatter(X_test[0, :], X_test[1, :], c="lightgray", s=10, alpha=0.5)

    for i, (center, radius) in enumerate(detected):
        circle = Circle(center, radius, fill=False, color=colors[i], linewidth=2)
        ax.add_patch(circle)

    ax.set_title(f"Noise: {noise}\nDetected: {len(detected)}/{len(centers)}")
    ax.set_aspect("equal")
    ax.grid(True, alpha=0.3)
    ax.set_xlim([-5, 6])
    ax.set_ylim([-5, 6])

plt.tight_layout()
plt.show()